### Es sólo la prueba de corrida sin vaciar el análisis al Notebook.

#### Which option should I run?
Options are:

0 = run all data and MC one after another

1 = run data only

2,3 = run MC samples only

In [1]:
int option=1; //option
cout << "Option: " << option;

Option: 1

#### Should I use PROOF? (will make things faster)
Options are:

0 = NO

1 = YES

In [2]:
int parallel=0; //option
cout << "Option: " << parallel;

Option: 0

In [3]:
%%cpp -d
#include "main_HZZAnalysis.cpp+"
//#include "HZZAnalysis.cpp+"

In [4]:
main_HZZAnalysis(parallel,option);

Starting analysis with process option: data 


Error in <TChain::SetBranchAddress>: unknown branch -> scaleFactor_TauTRIGGER
Error in <TChain::SetBranchAddress>: unknown branch -> scaleFactor_DiTauTRIGGER
Error in <TChain::SetBranchAddress>: unknown branch -> trigT
Error in <TChain::SetBranchAddress>: unknown branch -> trigDT
Error in <TTree::SetBranchStatus>: unknown branch -> scaleFactor_TauTRIGGER
Error in <TTree::SetBranchStatus>: unknown branch -> scaleFactor_DiTauTRIGGER
Error in <TTree::SetBranchStatus>: unknown branch -> trigT
Error in <TTree::SetBranchStatus>: unknown branch -> trigDT
Error in <TTree::SetBranchStatus>: unknown branch -> scaleFactor_TauTRIGGER
Error in <TTree::SetBranchStatus>: unknown branch -> scaleFactor_DiTauTRIGGER
Error in <TTree::SetBranchStatus>: unknown branch -> trigT
Error in <TTree::SetBranchStatus>: unknown branch -> trigDT
Error in <TTree::SetBranchStatus>: unknown branch -> scaleFactor_TauTRIGGER
Error in <TTree::SetBranchStatus>: unknown branch -> scaleFactor_DiTauTRIGGER
Error in <TTree::Se

Writting with name option: data 
